# **Primer análisis de datos extraidos**

#### Incluimos las librerías necesarias para analizar los datos

In [2]:
import pandas as pd
import json
from collections import defaultdict
import re
from datetime import datetime, timedelta

#### Leemos los datos del JSON y lo guardamos en la variable *datos*

In [3]:
datos = pd.read_json('traces_Articoding_Escolapias.json')
jsonData = json.load(open('traces_Articoding_Escolapias.json'))

# Comprobamos si el objeto que miramos es un nivel

In [4]:
def get_tipo(tipo):
    return tipo.rsplit('/',1)[-1]

def es_nivel(url):
    subtipo = get_tipo(url)
    return subtipo == 'level'


#EJ:
prueba = datos.head(5) 

for caso in prueba.to_numpy():
    tipo = caso[4]['definition']['type']
    nivel = es_nivel(tipo)
    print("->")
    print(caso[4]['id'])
    if nivel:
        print(get_tipo(caso[4]['definition']['type'])+"\nEs un nivel")
    else:
        print(get_tipo(caso[4]['definition']['type'])+"\nNo es nivel")


->
https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/tip_0_bienvenido
dialog-fragment
No es nivel
->
https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/articoding
serious-game
No es nivel
->
https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/articoding
serious-game
No es nivel
->
https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/categories_main
screen
No es nivel
->
https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/tutorials_general
screen
No es nivel


## Obtenemos el codigo de inicio y fin de nivel

In [5]:

prueba = datos.head(20)

tiempos = set()   
#Buscamos el nombre de los tiempos de inicio y fin, los cuales se encuentran en 'verb'
for time in prueba['verb']:
    tiempos.add(time['id'])
#Guardamos el link de inicio y fin
print(tiempos)
initialized = "http://adlnet.gov/expapi/verbs/initialized"
completed = "http://adlnet.gov/expapi/verbs/completed"
print('Initialized: '+ initialized)
print('Completed: '+ completed)


{'http://adlnet.gov/expapi/verbs/progressed', 'http://adlnet.gov/expapi/verbs/completed', 'http://adlnet.gov/expapi/verbs/initialized', 'https://w3id.org/xapi/seriousgames/verbs/accessed'}
Initialized: http://adlnet.gov/expapi/verbs/initialized
Completed: http://adlnet.gov/expapi/verbs/completed


#### Lista con los tiempos de cada usuario en cada nivel

Guarda los tiempos que ha tardado cada usuario en terminar cada nivel, en caso de completarlo varias veces, guarda todos los tiempos en una lista

In [6]:


def get_completion_time_list(datos):
    #verb nos dice si se ha iniciado o completado un nivel
    #Buscamos guardar en un diccionario de usuarios diccionarios de niveles con los tiempos de ejecución
    userList = defaultdict(lambda: defaultdict(list))
    #ketTuple nos permite saber si el usuario ya ha jugado ese nivel con anterioridad
    iniTime = defaultdict()
    tam = len(datos)
    for elemento in datos.to_numpy():
        #Comprobamos que el objeto sea un nivel
        if get_tipo(elemento[4]['definition']['type']) == 'level':
            verb = elemento[2]['id']
            name = elemento[0]['name']
            timestamp = elemento[5]
            level = elemento[4]['id']

            #Si es un nuevo nivel marcamos el tiempo de inicio
            if (name,level) not in iniTime or verb == initialized:
                iniTime[(name,level)] = timestamp
            else:
                userList[name][level].append(timestamp-iniTime[(name,level)]) 
                iniTime.pop((name,level))

    return userList


            
time_list = get_completion_time_list(datos)
#Mostramos los tiempos que tiene el usuario 'gqoj' cada vez que completó el primer tutorial
for element in time_list['gqoj']["https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/tutorials_1"]:
    print(str(element))


0 days 00:00:16.992000
0 days 00:00:48.227000
0 days 00:00:32.308000


## Obtenemos las tarjetas usadas durante un nivel

In [5]:
#Obtiene informacion de las tarjetas en cada nivel
class Bloque():
    def __init__(self,tipo,codigo,nivel,accion):
        self.codigo = codigo
        self.tipo = tipo
        self.nivel = nivel
        self.accion = accion
    
    def __init(self,bloque):
        self.codigo = bloque["code"]
        self.tipo = bloque["block_type"]
        self.nivel = bloque["level"]
        self.accion = bloque["create"]

def nivel_principal(verb,nivel,erCompleted):
    erAnomalyLevels = re.compile(r'\btutorials|editor_level|tip|procedures')
    return erAnomalyLevels.search(nivel)


def obtener_tarjetas(datos):
    #Recorre el array y guarda los bloques de tarjetas de los niveles completados
    #Las tarjetas se manipulan cuando:
    
    erLevel = re.compile(r'\blevel$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erProgressed = re.compile(r'\bprogressed$\b')
    erInteracted = re.compile(r'\binteracted$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erStartButton = re.compile(r'\bstart_button$\b')
    erIdLevel = re.compile(r'/')
    tarjetas = list()
    antId = ''
    antLevel = ''
    infoBloque = ""
    encontrado = False
    for evento in datos:
        obj = evento["object"]["definition"]["type"]
        objectId = evento["object"]["id"]
        verb = evento["verb"]["id"]
        name = evento["actor"]["name"]
        if erLevel.search(obj) and encontrado:
            levelCode = erIdLevel.split(objectId)[-1]
            if erCompleted.search(verb):
                if evento["result"]["success"]:
                    #Un nivel completado tiene 3 eventos
                    #Primero uno con el codigo de ejecución (Initialized)
                    #Un segundo evento con flag Progressed
                    #Un tercer evento con el resultado del nivel (success,stars,etc.)
                    tarjetas.append(code)
                    encontrado = False
        
        #Para evitar que sobreescriba el codigo antes de terminar el nivel
        elif "result" in evento and erGameObject.search(obj) and erInteracted.search(verb):
            if "extensions" in evento["result"] and not erProgressed.search(verb) and not nivel_principal(verb,objectId,erCompleted):
                if "code" in evento["result"]["extensions"]:
                    code = evento["result"]["extensions"]["code"]
                    antId = name
                    antLevel = objectId
                    encontrado = True
        

    return tarjetas


pd.DataFrame(obtener_tarjetas(jsonData))